This code implements a directed graph.

In [1]:
class Graph:
    def __init__(self, directed=False):
        self.graph = {}
        self.directed = directed

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []

    def add_edge(self, src, dest):
        if src not in self.graph:
            self.add_vertex(src)
        if dest not in self.graph:
            self.add_vertex(dest)
        self.graph[src].append(dest)
        if not self.directed:
            self.graph[dest].append(src)

    def remove_edge(self, src, dest):
        if src in self.graph:
            if dest in self.graph[src]:
                self.graph[src].remove(dest)
        if not self.directed:
            if dest in self.graph and src in self.graph[dest]:
                self.graph[dest].remove(src)

    def remove_vertex(self, vertex):
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    self.graph[adj].remove(vertex)
            # Remove the vertex entry
            del self.graph[vertex]

    def get_adjacent_vertices(self, vertex):
        if vertex in self.graph:
            return self.graph[vertex]
        else:
            return []

    def __str__(self):
        return str(self.graph)

# Example usage:
g = Graph(directed=True)
g.add_vertex('A')
g.add_vertex('B')
g.add_edge('A', 'B')
g.add_edge('A', 'C')
print(g)

{'A': ['B', 'C'], 'B': [], 'C': []}


What improvements or changes would you need to implement to make the code ready for production settings? Is there any missing functionality? Or any security vulnerabilities to consider?

<div class='alert alert-block alert-info'>
<b>Asked to LLM :</b><br>

You are an expert software developer or site reliability engineer, and your job is to ensure that this code runs effectively, quickly, at scale, and securely, Please profile it, and find any issues that need to be fixed or updated.

In [8]:
import threading
from collections import defaultdict

class Graph:
    def __init__(self, directed=False):
        self.graph = defaultdict(set)  # Use set to prevent duplicate edges
        self.directed = directed
        self.lock = threading.RLock()  # Thread-safe modifications

    def add_vertex(self, vertex):
        if not isinstance(vertex, (int, str)):
            raise ValueError("Vertex must be hashable (int or str).")
        with self.lock:
            self.graph[vertex]  # Initialize vertex in the graph

    def add_edge(self, src, dest, weight=None):
        if not isinstance(src, (int, str)) or not isinstance(dest, (int, str)):
            raise ValueError("Vertices must be hashable (int or str).")
        with self.lock:
            self.graph[src].add((dest, weight))
            if not self.directed:
                self.graph[dest].add((src, weight))

    def remove_edge(self, src, dest):
        with self.lock:
            if src in self.graph and (dest, None) in self.graph[src]:
                self.graph[src].discard((dest, None))
            elif src in self.graph:
                # Remove specific weighted edges
                self.graph[src] = {edge for edge in self.graph[src] if edge[0] != dest}
            if not self.directed and dest in self.graph:
                self.graph[dest].discard((src, None))

    def remove_vertex(self, vertex):
        with self.lock:
            if vertex in self.graph:
                # Remove all incoming edges
                for adj in self.graph:
                    self.graph[adj] = {edge for edge in self.graph[adj] if edge[0] != vertex}
                # Remove the vertex itself
                del self.graph[vertex]

    def get_adjacent_vertices(self, vertex):
        return list(self.graph[vertex]) if vertex in self.graph else []

    def __str__(self):
        with self.lock:
            return '\n'.join(f"{v}: {list(adj)}" for v, adj in self.graph.items())

    def bfs(self, start):
        """Breadth-First Search Traversal"""
        visited = set()
        queue = [start]
        result = []

        while queue:
            vertex = queue.pop(0)
            if vertex not in visited:
                visited.add(vertex)
                result.append(vertex)
                queue.extend(neigh[0] for neigh in self.graph[vertex] if neigh[0] not in visited)
        return result

    def dfs(self, start, visited=None):
        """Depth-First Search Traversal"""
        if visited is None:
            visited = set()
        visited.add(start)
        result = [start]

        for neighbor in self.graph[start]:
            if neighbor[0] not in visited:
                result.extend(self.dfs(neighbor[0], visited))
        return result

# Example usage:
g = Graph(directed=True)
g.add_vertex('A')
g.add_vertex('B')
g.add_edge('A', 'B', weight=5)
g.add_edge('A', 'C', weight=10)
print(g)
print("BFS from A:", g.bfs('A'))
print("DFS from A:", g.dfs('A'))


A: [('C', 10), ('B', 5)]
B: []
BFS from A: ['A', 'C', 'B']
DFS from A: ['A', 'C', 'B']


In [9]:
import threading

def test_graph():
    g = Graph(directed=True)

    # 1️⃣ Adding duplicate vertices
    print("\n🔹 Test: Adding duplicate vertices")
    g.add_vertex('A')
    g.add_vertex('A')  # Should not create a duplicate
    assert len(g.graph) == 1, "Duplicate vertex was added!"

    # 2️⃣ Adding duplicate edges
    print("\n🔹 Test: Adding duplicate edges")
    g.add_edge('A', 'B')
    g.add_edge('A', 'B')  # Should not add duplicate
    assert len(g.graph['A']) == 1, "Duplicate edge was added!"

    # 3️⃣ Adding self-loops
    print("\n🔹 Test: Adding self-loops")
    g.add_edge('A', 'A')  # Self-loop
    assert ('A', None) in g.graph['A'], "Self-loop not added correctly!"

    # 4️⃣ Removing non-existent edge
    print("\n🔹 Test: Removing non-existent edge")
    g.remove_edge('A', 'C')  # No such edge should exist
    assert len(g.graph['A']) == 2, "Non-existent edge removal failed!"

    # 5️⃣ Removing non-existent vertex
    print("\n🔹 Test: Removing non-existent vertex")
    g.remove_vertex('X')  # No such vertex
    assert len(g.graph) == 1, "Removing non-existent vertex caused issues!"

    # 6️⃣ Removing vertex with edges
    print("\n🔹 Test: Removing vertex with edges")
    g.add_edge('B', 'C')
    g.remove_vertex('A')
    assert 'A' not in g.graph, "Vertex A was not removed!"
    assert 'A' not in g.graph['B'], "Edges to A were not removed!"

    # 7️⃣ Graph traversal (BFS, DFS) on disconnected graph
    print("\n🔹 Test: Graph traversal on disconnected graph")
    g.add_vertex('X')
    bfs_result = g.bfs('X')
    dfs_result = g.dfs('X')
    assert bfs_result == ['X'], "BFS on disconnected graph failed!"
    assert dfs_result == ['X'], "DFS on disconnected graph failed!"

    # 8️⃣ Graph traversal (BFS, DFS) on cyclic graph
    print("\n🔹 Test: Graph traversal on cyclic graph")
    g.add_edge('X', 'Y')
    g.add_edge('Y', 'Z')
    g.add_edge('Z', 'X')  # Cycle
    assert len(g.bfs('X')) == 3, "BFS on cyclic graph failed!"
    assert len(g.dfs('X')) == 3, "DFS on cyclic graph failed!"

    # 9️⃣ Handling large graphs
    print("\n🔹 Test: Handling large graphs (100K nodes, 200K edges)")
    large_graph = Graph(directed=False)
    for i in range(100000):
        large_graph.add_vertex(i)
    for i in range(100000):
        large_graph.add_edge(i, (i+1) % 100000)  # Circular edges
    assert len(large_graph.graph) == 100000, "Large graph failed!"
    assert len(large_graph.graph[0]) == 2, "Edge insertion failed for large graph!"

    # 🔟 Concurrency safety test
    print("\n🔹 Test: Concurrency safety")
    def add_edges():
        for i in range(100):
            g.add_edge(i, i + 1)

    def remove_edges():
        for i in range(100):
            g.remove_edge(i, i + 1)

    thread1 = threading.Thread(target=add_edges)
    thread2 = threading.Thread(target=remove_edges)

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

    print("\n✅ All tests passed successfully!")

# Run the test suite
test_graph()



🔹 Test: Adding duplicate vertices

🔹 Test: Adding duplicate edges

🔹 Test: Adding self-loops

🔹 Test: Removing non-existent edge

🔹 Test: Removing non-existent vertex

🔹 Test: Removing vertex with edges

🔹 Test: Graph traversal on disconnected graph

🔹 Test: Graph traversal on cyclic graph

🔹 Test: Handling large graphs (100K nodes, 200K edges)

🔹 Test: Concurrency safety

✅ All tests passed successfully!
